In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
from scipy.stats import threshold

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pytpc

In [ ]:
padmap = pd.read_csv('/Users/josh/Dropbox/routing/Lookup20141208.csv', header=None,
                     names=('cobo', 'asad', 'aget', 'channel', 'pad'), index_col=-1)

In [ ]:
pad_lookup = {(cobo, asad, aget, ch) : pad for pad, cobo, asad, aget, ch in padmap.itertuples()}

In [ ]:
gfile_paths = glob.glob('/Users/josh/Documents/Data/Alphas-Dec14/run_0225/*.graw')

In [ ]:
gfiles = [pytpc.grawdata.GRAWFile(p) for p in gfile_paths]

In [ ]:
evt = pytpc.grawdata.merge_frames(gfiles, 100)
evt.traces['pad'] = [pad_lookup.get(tuple(a), 20000) for a in evt.traces[['cobo', 'asad', 'aget', 'channel']]]
# evt.traces = np.delete(evt.traces, np.where(evt.traces['pad'] == 20000))
evt.traces['data'] = (evt.traces['data'].T - np.median(evt.traces['data'], axis=1)).T
evt.traces['data'] = threshold(evt.traces['data'], 20.)

In [ ]:
np.unique(evt.traces[np.where(evt.traces['pad'] == 20000)]['channel'])

In [ ]:
pytpc.pad_plot(evt.hits());

In [ ]:
for d in evt.traces['data']:
    plt.plot(d)
plt.plot(evt.traces['data'].sum(0), 'k--')